In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from collections import defaultdict
pd.options.display.max_rows = 100
pd.options.display.max_columns = None



### Success Defined Through Revenue

- Find the highest profitting films 
    - genres 
    - actors 
    - directors 
    - budget 
    - word title
    - length 

### 23m Data (profit/revenue)

In [ ]:
ratings_25m = pd.read_csv('../../data/m_23/movies_metadata.csv')
print(ratings_25m.shape)
print(ratings_25m.columns)
ratings_25m.head(3)
#revenue_df['profit'] = round(pd.to_numeric(revenue_df.revenue) - pd.to_numeric(revenue_df.budget), 2)
col = ['imdb_id', 'title', 'release_date', 'budget', 'revenue', 'genres', 'adult', 'overview']
revenue_df = ratings_25m[[i for i in col]]

In [ ]:

def budget_numeric(df): 
    new_col = []
    new_df = df[[i for i in df.columns.to_list() if i!= 'budget']]
    col_val = np.array(df[f'budget'])
    for item in col_val: 
        if '.jpg' in item: 
            new_col.append(None)
        else: 
            new_col.append(float(item))
    new_df['budget'] = new_col
    return new_df

def profits(df):
    new_df = df
    new_df['profit'] = round(new_df.revenue - new_df.budget,2)
    return new_df

def dates_func(x):
    if type(x) != str: 
        return None
    else: 
        return x.split('-')[0]
def dates(df): 
    dates = df.release_date 
    df.drop(['release_date'], axis = 1, inplace = True)
    df['date'] = dates.map(dates_func)
    return df

def genres1(df):
    genre_list = df.genres.values
    genres = defaultdict(int)
    new_genres = []
    for i in tqdm(genre_list): 
        if i[0] == '[' and i[1] == ']': 
            new_genres.append(None)
            continue 
        first = i.split('}, ')[0]
        name_loc = first.find("'name': ")
        new_g= first[name_loc +9:]
        new_g = new_g[:new_g.find("'")].strip()
        new_genres.append(new_g)
    df['genre'] = new_genres
    df.drop('genres', axis =1, inplace = True)


        
    df.to_csv('../UsedData/23mRevenueTEST.csv', index = False)


    return df

def genres2(df):
    genre_list = df.genres.values
    genres = defaultdict(int)
    new_genre = []
    for i in genre_list: 
        if i[0] == '[' and i[1] == ']': 
            continue 
        dic_split = i.split(', ')
        for d in dic_split: 
            name_loc = d.find("'name': ")
            new_d= d[name_loc +9:]
            new_d = new_d[:new_d.find("'")].strip()
            #new_d = new_d[:new_d.find("'")].strip()
            genres[new_d]+=1
    for g in tqdm(genres.keys()): 
        df[f'{g}'] = [0 for i in range(len(df))]
    for pos in tqdm(range(len(df))): 
        i = df.iloc[pos].genres
        if i[0] == '[' and i[1] == ']': 
            new_genre.append(None)
            continue 
        dic_split = i.split(', ')
        for idx, d in enumerate(dic_split): 
            name_loc = d.find("'name': ")
            new_d= d[name_loc +9:]
            new_d = new_d[:new_d.find("'")].strip()
            if idx == 0:
                new_genre.append(new_d)
            df.iloc[pos, df.columns.get_loc(f'{new_d}')] = 1


    #print(len(df), len(new_genre))        
    df['genre'] = new_genre
    df.drop('genres', axis =1, inplace = True)
    df.to_csv('../UsedData/23mRevenueNEW.csv', index = False)
    return df

        


    

cleaned_23m = budget_numeric(revenue_df)
cleaned_23m = profits(cleaned_23m)
cleaned_23m = dates(cleaned_23m)
cleaned_23m_one_hot = genres2(cleaned_23m)                 
cleaned_23m = genres1(cleaned_23m)

In [ ]:
cleaned_23m

## MovieLens Data for Genre and Ratings

In [ ]:
def dates(x):
    num = '1 2 3 4 5 6 7 8 9 0'.split()
    if x.strip()[-1] != ')' or x.strip()[-2] not in num: 
        return None
    else: 
        return x.strip()[-5:-1]

def title(x): 
    num = '1 2 3 4 5 6 7 8 9 0'.split()
    x = x.strip()
    if x[-1] != ')' or x[-2] not in num: 
        return x
    else: 
        return x[:-5]

def sort_genres(df):
    genre_dict = defaultdict(int)
    genre_list = np.array(df.genres)
    for genre in tqdm(genre_list): 
        g_split = genre.split('|')
        for i in g_split:
            genre_dict[i]+=1
    for g in tqdm(genre_dict.keys()): 
        df[f'{g}'] = [0 for i in range(len(df))]
    for pos in tqdm(range(len(df))): 
        row = df.iloc[pos]
        g_split = row['genres'].split('|')
        for i in g_split: 
            df.iloc[pos, df.columns.get_loc(f'{i}')] = 1
    df.to_csv('../UsedData/MovieLensMoviesCleaned.csv', index = False)
    return df

In [ ]:
movie_lens = pd.read_csv('../../data/MovieLens/movies.csv')
print(movie_lens.shape)
print(movie_lens.columns)
movie_lens['date'] = movie_lens.title.map(dates)
title_s = movie_lens['title']
movie_lens.drop(['title'], axis = 1, inplace = True)
movie_lens['title'] = title_s.map(lambda x: x[:-6].strip())
movie_lens = movie_lens[['movieId', 'title', 'date', 'genres']].sort_values(by = ['date'], ascending = False)
movie_lens.head(5)
movie_lens_sorted = sort_genres(movie_lens)

In [ ]:
movie_lens_movies = movie_lens_sorted

In [ ]:
movie_lens_movies

### MovieLens Ratings

In [ ]:
cd ../data/m_23


In [ ]:
#movie_lens_ratings = pd.read_csv('data/MovieLens/ratings.csv')

ratings_23m = pd.read_csv('../../data/m_23/ratings.csv')

### Merge Ratings, revenues, movies

In [ ]:
def agg_ratings(df):
    grouped_df = df.groupby('movieId')
    rating_avg = grouped_df.rating.mean().values
    sum_interactions = grouped_df.rating.count()
    movie_id = sum_interactions.index
    sum_inter = sum_interactions.values
    new_df = pd.DataFrame({'movieId': movie_id, 'AvgRating': rating_avg, 'NumInteractions': sum_interactions})
    new_df.to_csv('../UsedData/MovieLensRatingsAgg.csv', index = False)
    return new_df.reset_index(drop = True)
    

    
    
    

def merge_df(revenue_df, movie_df, rating_df):
    
    #merge revenue and movie first 
    merge_df = pd.merge(movie_df, revenue_df, how = 'inner', on = ['title', 'date']).dropna(subset = ['title'])
    
    rating_df = agg_ratings(rating_df)

    merge_df = pd.merge(merge_df, rating_df, how = 'inner', on = ['movieId']).drop(['genres'], axis = 1)
    
    merge_df.to_csv('../UsedData/Merged_ratings_movies_revenue_New.csv', index = False)
    return merge_df
    
    
 

merged_df = merge_df(cleaned_23m, movie_lens_movies, ratings_23m)

In [ ]:
merged_df